In [1]:
# imports
import os
import pickle
import time
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# changing directory
# Run this cell only once!!
path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)

In [3]:
# get the data

# general information
continents = pd.read_csv('data/continents.csv')
countries = pd.read_csv("data/countries.csv")

# timeseries
global_ts = pd.read_csv("data/global_ts.csv")
country_confirmed = pd.read_csv("data/country_confirmed.csv")
country_deaths = pd.read_csv("data/country_deaths.csv")
country_recovered = pd.read_csv("data/country_recovered.csv")
vac_global = pd.read_csv("data/vac_global.csv")
vac_country = pd.read_csv("data/vac_country.csv")

# pickle file has only data for text and nothing for chart or analysis
with open("data/global_update.pkl", "rb") as file:
    global_update = pickle.load(file)


In [4]:
# updating index for all dataframes
continents.index = continents['continent']
continents.drop(columns=['continent'], inplace=True)
country_confirmed.index = country_confirmed['country']
country_confirmed.drop(columns=['country'], inplace=True)
country_deaths.index = country_deaths['country']
country_deaths.drop(columns=['country'], inplace=True)
country_recovered.index = country_recovered['country']
country_recovered.drop(columns=['country'], inplace=True)
vac_country.index = vac_country['country']
vac_country.drop(columns=['country'], inplace=True)

# vac_global and golbal_ts are a single time series so no need to change their index

In [5]:
def combined_country(country,confirmed_df, recovered_df, deaths_df, vac_df):
    comb_df = pd.DataFrame()
    comb_df["confirmed"] = confirmed_df.loc[country]
    comb_df["deaths"] = deaths_df.loc[country]
    comb_df["recovered"] = recovered_df.loc[country]
    comb_df["vaccined"] = vac_df.loc[country]
    return comb_df


In [6]:
# Example
combined_country("India", country_confirmed, country_recovered, country_deaths, vac_country)

,confirmed,deaths,recovered,vaccined
1/22/20,0,0,0,NaN
1/23/20,0,0,0,NaN
1/24/20,0,0,0,NaN
1/25/20,0,0,0,NaN
1/26/20,0,0,0,NaN
...,...,...,...,...
9/1/21,32857937,439529,0,661576916.0
9/2/21,32903289,439895,0,669775627.0
9/3/21,32945907,440225,0,675654935.0
9/4/21,32988673,440533,0,677744969.0


In [7]:
global_ts

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,8/28/21,8/29/21,8/30/21,8/31/21,9/1/21,9/2/21,9/3/21,9/4/21,9/5/21,category
0,557,655,941,1433,2118,2927,5578,6167,8235,9927,...,215991185,216433910,217119865,217708315,218431264,219062869,219821446,220282354,220689611,cases
1,17,18,26,42,56,82,131,133,171,213,...,4494714,4501435,4510359,4519444,4531889,4541463,4553701,4561159,4567350,deaths
2,30,32,39,42,56,65,108,127,145,225,...,0,0,0,0,0,0,0,0,0,recovered
